# Databases and MySQL

In [ ]:
# Setting Up
import mysql.connector
# if import fails, 'conda install mysql-connector-python' to install (from a conda prompt)
import pandas as pd

## Flight-Airplane-PassengerCount

In [ ]:
# read from csv
flights = pd.read_csv("../data/flights.csv", parse_dates=['DepDate', 'ArrDate'])
flights.head()

## People-Flight-Airplane

In [ ]:
# read from csv
passengers = pd.read_csv("../data/passengers.csv", parse_dates=['DepDate', 'ArrDate'])
passengers.head()

## Connecting to and querying the database

In [ ]:
# connect to the database
cnx = mysql.connector.connect(
    user="insy6500", 
    password="insy6500", 
    host="localhost", 
    database="insy6500"
    )
print ("Successfully connected....")

In [ ]:
# List the tables using a cursor object
print ("Tables:")
cursor = cnx.cursor()
# this is the SQL query
query = ("show tables")
cursor.execute(query)
for (db) in cursor:
    print ("\t{}".format(db[0]))
cursor.close()
print ("Database closed.")

## Define Some Helper Functions

In [ ]:
#
#  Single value query - returns the result of a single-value query
#
def single_value(query):
    cursor = cnx.cursor()
    cursor.execute(query)
    db = cursor.fetchall()
    cursor.close()
    return(db[0][0])

#
#  Recordset query - returns the result of a recordset query
#
def recordset(query):
    cursor = cnx.cursor()
    cursor.execute(query)
    db = cursor.fetchall()
    cursor.close()
    return(db)

In [ ]:
qry = "select count(*) from passenger"
print("{:,d} passenger records.".format(single_value(qry)))

In [ ]:
qry = "select * from airplane"
print("Airplane records: {}".format(recordset(qry)))

In [ ]:
rs = recordset(qry)

In [ ]:
for r in rs:
    print(r[0])

## Using Pandas

In [ ]:
# Sample query to a Pandas dataframe
qry = "select * from airplane"
print ("Query: {:}".format(qry))
pdPlanes = pd.read_sql(qry, cnx)
pdPlanes

In [ ]:
# Query to an anonymous dataframe
pd.read_sql("show tables", cnx)

In [ ]:
qry = "select * from passenger limit 5"
pd.read_sql(qry, cnx)

In [ ]:
# Define a query helper function
# Assumes that the connection cnx has already been established
def pquery(qry):
    p = pd.read_sql(qry,cnx)
    print("Found {:,d} records.".format(len(p)))
    return p

### Standard SELECT statements

In [ ]:
qry = "select * from booking"
p = pquery(qry)
p.head()

In [ ]:
qry = "select * from booking limit 100"
p = pquery(qry)
p.head()

In [ ]:
p = pquery("select distinct FlightNumber from flight")
p.head(10)

In [ ]:
pquery("select * from flight order by DepartureTime limit 10")

In [ ]:
pquery("select * from flight where day(DepartureDate) = 5 order by DepartureTime")

In [ ]:
pquery("select * from flight where day(DepartureDate) = 5 and Destination = 'Atlanta' order by DepartureTime")

### Aggregation and Grouping

In [ ]:
pquery("describe skus")

In [ ]:
pquery("select sum(qty_on_hand) from skus")

In [ ]:
pquery("select sum(qty_on_hand * weight) from skus")

## Querying Multiple Tables

In [ ]:
# Subqueries
pquery("select * from artist where artist_id not in (select distinct artist_id from track)")
# add an artist with no albums and rerun

In [ ]:
# Simply joins
pquery("select artist_name, album_name from artist inner join album using (artist_id) order by artist_name")

### Examples - Was a Homework Assignment last time

In [ ]:
# Simple inner join
# 1.  Artists and their albumns --- artist_name, album_name


In [ ]:
# 2.  Artists and tracks


In [ ]:
# 3. Artists and the number of tracks they have


In [ ]:
# 4. Track anm,e, date/time played for all tracks that have been played


In [ ]:
# 5. Including songs that haven't been played


In [ ]:
# 6. Track name, times played, total time played for all songs that have been played.


In [ ]:
# 7. All airplanes and the flights that each has flown


In [ ]:
# 8. All airplanes and the number of flights that each has flown.

In [ ]:
# 9.  All flights with the plan capacities and number of passengers booked on each


In [ ]:
# 10. Flights originating in Atlanta that have remaining capacity
